In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from tqdm.notebook import tqdm
from keras.layers import Dense
from numpy import pi

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

def createTraj():
    fhat = Sequential()
    fhat.add(Dense(50, activation="sigmoid", input_dim=1))
    fhat.add(Dense(1))
    return fhat

x = createTraj()
y = createTraj()
velocity = createTraj()
theta = createTraj()
thrust = createTraj()
moment = createTraj()

initial_time = 0
final_time = 1
number_of_points = 20
normalize_time = tf.cast(tf.linspace(initial_time,final_time,number_of_points),dtype=tf.float32)

Num GPUs Available:  0
[]


In [3]:
def errors(t):
  with tf.GradientTape(persistent=True) as tape1:
    tape1.watch(t)
    with tf.GradientTape(persistent=True) as tape2:
        tape2.watch(t)
        X = x(t)
        Y = y(t)
        TH = theta(t)
        V = velocity(t)
    
    Xd = tape2.gradient(X, t)
    Yd = tape2.gradient(Y, t)

  Xdd = tape1.gradient(Xd, t)
  Ydd = tape1.gradient(Yd, t)

  # Compute the derivatives
  THd = tape1.gradient(TH, t)
  Vd = tape1.gradient(V, t)

  # Error in dynamics
  e_xdot = tf.reduce_sum((tf.reshape(Xdd, shape=(number_of_points, 1)) - Vd * tf.cos(TH)) ** 2)
  e_ydot = tf.reduce_sum((tf.reshape(Ydd, shape=(number_of_points, 1)) - Vd * tf.sin(TH)) ** 2)

  # Error in initial condition
  x0 = 0
  y0 = 0
  th_0 = 0
  x_dot_0 = 0
  y_dot_0 = 0
  th_dot_0 = 0
  eIC = (X[0] - x0) ** 2 + (Y[0] - y0) ** 2 + (TH[0]) ** 2 + V[0] ** 2  # start from (x0,y0) at rest and pointing east

  # Error in final condition
  xf = 1
  yf = 1
  th_f = 0
  x_dot_f = 0
  y_dot_f = 0
  th_dot_0 = 0
  eFC = (X[-1] - xf) ** 2 + (Y[-1] - yf) ** 2 + (TH[-1] - np.pi) ** 2 + V[-1] ** 2  # end at (xf,yf) at rest and pointing west

  return e_xdot + e_ydot + 10 * eIC + 10 * eFC